# Opening Lead

This is a tutorial for how to use the opening lead engine.

In [11]:
!pip install --upgrade tensorflow[and-cuda]
!pip install -q huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 13.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 26.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 55.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 28.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 21.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 MB 20.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 85.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 78.6 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of tensorflow[and-cuda] to determine which version is comp

In [12]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotLead
from sample import Sample
import conf

In [13]:
#models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
#sampler = Sample.from_conf(conf.load('./config/default.conf'), False)  # Load sampling strategies
import huggingface_hub
import os
lead_id="AIBridgeEngine/Ben-3B-Lead-v0.1"
bidding_id="AIBridgeEngine/Ben-3B-Bidding-v0.1"
biddinginfo_id="AIBridgeEngine/Ben-3B-Biddinginformation-v0.1"
singledummy_id="AIBridgeEngine/Ben-3B-Singledummy-v0.1"
folder="hfmodels"
huggingface_hub.snapshot_download(repo_id=lead_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=bidding_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=biddinginfo_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=singledummy_id, local_dir=folder)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

'/home/jovyan/ben/hfmodels'

In [14]:
from nn.leader import Leader
from nn.bid_info import BidInfo
from nn.bidder import Bidder
from nn.lead_singledummy import LeadSingleDummy
lead = Leader(f'{folder}/lead-1000000')
biddinginfo = BidInfo(f'{folder}/binfo-100000')
bidder = Bidder("", f'{folder}/bidding-100000')
sd_model = LeadSingleDummy(f'{folder}/lr3-1000000')
models = Models(bidder, biddinginfo, lead, sd_model, None, 0.1, 0.05)
sampler = Sample(0.01, 0.1, 0.03, 0.05, 64, 5000, 5000, 128, 100, False)

INFO:tensorflow:Restoring parameters from hfmodels/lead-1000000
INFO:tensorflow:Restoring parameters from hfmodels/binfo-100000
INFO:tensorflow:Restoring parameters from hfmodels/bidding-100000
INFO:tensorflow:Restoring parameters from hfmodels/lr3-1000000


In [15]:
# both vulnerable. you are sitting North as dealer and you hold
hand = 'T54.QT87.A98.943'

# the auction goes:
auction = ["1C","1D","1S","2D","X","PASS","2S","PASS","3C","PASS","3N","PASS","4S",'PASS', 'PASS', 'PASS']

# what to lead?

lead_bot = BotLead([True, True], hand, models, -1, -1, models.lead_threshold, sampler, False)
lead = lead_bot.find_opening_lead(auction)

In [16]:
lead.card

DA

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [17]:
lead.to_dict()['candidates']

[{'card': 'DA',
  'insta_score': 0.1587,
  'expected_tricks': 10.16,
  'p_make_contract': 0.69},
 {'card': 'HT',
  'insta_score': 0.0649,
  'expected_tricks': 10.24,
  'p_make_contract': 0.7},
 {'card': 'Hx',
  'insta_score': 0.6122,
  'expected_tricks': 10.3,
  'p_make_contract': 0.7}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [18]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['AQ.K9xx.Qx.K8xxx Kxx.AJxx.Jxxxx.Q J98xx.x.KTx.AJTx',
 'KQx.A9xx..KQT8xx Ax.xx.KTxxxx.Jxx J98xx.KJx.QJxx.A',
 'AK8.9xxx..AQJTxx xx.AKJ.QJTxxx.8x QJ9xx.xx.Kxxx.Kx',
 'Q9x.Kxx.K.AQ8xxx 8xx.Axx.JTxxx.Kx AKJx.J9x.Qxxx.JT',
 'AKQ.Jx.x.AQJ8xxx xxx.Kxxx.QJTxx.K J98x.A9x.Kxxx.Tx',
 '9x.Axx.x.AKQJTxx AQ8.J9x.QJTxx.8x KJxxx.Kxx.Kxxx.x',
 'Q98.AKJx.x.AKJ8x AJx.9x.KTxxxxx.x Kxxx.xxx.QJ.QTxx',
 'Q8x.xx.KJx.AKQTx K.A9xx.QTxxx.Jxx AJ9xxx.KJx.xx.8x',
 'Qxx.K9xx.x.AKQ8x K.AJxx.QJTxxx.Tx AJ98xx.x.Kxx.Jxx',
 'AJx.Axxx.x.AQxxx Qx.J9.JTxxxx.KT8 K98xx.Kxx.KQx.Jx',
 'AKxx.Axx..AQJTxx J8.KJx.QJxxxx.xx Q9xx.9xx.KTxx.K8',
 'AQJx.KJ9x.x.KQJx 8.xxx.KQJxxx.Txx K9xxx.Ax.Txx.A8x',
 'AKJ.xx.Txx.KQJTx Q9x.A9xx.KQxxx.x 8xxx.KJx.Jx.A8xx',
 'A8x.Kxx.T.AKQ8xx x.AJ9x.Kxxxx.Jxx KQJ9xx.xx.QJxx.T',
 'AQx.AKJ9.xx.Jxxx 8.xx.KQJxxxx.AQT KJ9xxx.xxx.T.K8x',
 'AQ8.Axxx.xx.AJxx Kx.Kx.QJTxxx.KTx J9xxx.J9x.Kx.Q8x',
 'K9x.AKJ.xx.AJTxx AQ.9xxx.KTxxx.Kx J8xxx.xx.QJx.Q8x',
 'Axx.KJ.QJx.AKJT8 J.Axxx.KTxxxx.Qx KQ98xx.9xx.x.xxx',
 'AQx.Kxx.